In [1]:
from pymongo import MongoClient, GEOSPHERE
from pandas.io.json import json_normalize
import pandas as pd
import os
import folium

In [2]:
client = MongoClient ('localhost', 27017)
db = client['companies']

### Se buscan las diferentes categorías, para ver cuales utilizamos

In [3]:
categorias = db.companies.distinct('category_code');
print(list(categorias))

['enterprise', 'web', 'software', 'news', 'network_hosting', 'games_video', 'mobile', 'music', 'social', 'search', 'messaging', 'advertising', 'photo_video', 'security', 'finance', 'ecommerce', 'travel', 'hardware', 'public_relations', 'other', 'real_estate', 'semiconductor', 'analytics', 'health', 'legal', 'sports', 'biotech', 'cleantech', 'education', 'consulting', 'transportation', None, 'hospitality', 'fashion', 'nonprofit', 'nanotech', 'automotive', 'design', 'manufacturing', 'government', 'local', 'medical']


### Se escoge una lista de categorías y los campos que vamos a obtener de la base de datos

In [ ]:
categorias = ['web', 'software', 'games_video', 'social', 'design', 'search']
campos = {'name': 1,
          'number_of_employees': 1,
          'category_code': 1,
          'acquisition': 1,
          'founded_year': 1,
          'funding_rounds': 1,
          'offices.latitude': 1,
          'offices.longitude': 1,
          'offices.city': 1,
          'offices.country_code': 1}

In [4]:
def buscar_oficinas(categorias, campos, anyo, mm, num_emp):
    '''
    Para realizar la búsqueda en la base de datos
    
    categorias: los item por los que filtrar
    campos: los campos a mostrar
    mm: par el número de empleados, si es mayor o menor $gt, $lt ...
    num_emp: número de empleados
    '''
    datos = db.companies.find({'$and':[{'category_code': {'$in':categorias}},
                                       {'number_of_employees': {mm: num_emp}},
                                       {'offices': {'$exists': True, '$not': {'$size': 0}}},
                                       {'deadpooled_year': None},
                                       {'founded_year':{'$gte': anyo}}]},
                              campos)
    return datos



### Se realizan dos búsquedas una para startup y otra para empresas grandes

In [ ]:
cursor_big = buscar_oficinas(categorias, campos, 1990, '$gt', 250)
cursor_startup = buscar_oficinas(categorias, campos, 2010, '$lt', 100)

#### Normalizamos el campo offices tanto para las startup como para las big

In [5]:
big_office_data = json_normalize(data = cursor_big, 
                             record_path ='offices', 
                             meta=['name', 'category_code', 'founded_year', 'number_of_employees']) 
big_office_data.head()

,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees
0,Menlo Park,USA,37.416050,-122.151801,Facebook,social,2004,5299
1,Dublin,IRL,53.344104,-6.267494,Facebook,social,2004,5299
2,New York,USA,40.755716,-73.979247,Facebook,social,2004,5299
3,San Francisco,USA,37.776805,-122.416924,Twitter,social,2006,1300
4,San Jose,USA,37.295005,-121.930035,eBay,web,1995,15000


In [7]:
startup_office_data = json_normalize(data = cursor_startup, 
                             record_path ='offices', 
                             meta=['name', 'category_code', 'founded_year', 'number_of_employees']) 
startup_office_data.head()

,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees
0,New York,USA,40.757929,-73.985506,PeekYou,search,2012,20
1,Berlin,DEU,52.501345,13.410907,headr,web,2012,8
2,Hannover,DEU,NaN,NaN,headr,web,2012,8
3,San Mateo,USA,37.566879,-122.323895,Fixya,web,2013,30
4,Palo Alto,USA,NaN,NaN,Simplicant,software,2012,10


### Se concatenan ambos dataframes y se eliminan los registros con nulos

In [8]:
data = pd.concat([startup_office_data, big_office_data])
data = data.reset_index()
data = data.dropna()
data.shape

(183, 9)

### Se juntan en un único campo tanto la longitud como la latitud, para pasarlo a mongo y utilizar geolocation

In [9]:
data['coordenadas'] = [[x, y] for x, y in zip(data['longitude'], data['latitude'])]
data.head()

,index,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees,coordenadas
0,0,New York,USA,40.757929,-73.985506,PeekYou,search,2012,20,"[-73.985506, 40.757929]"
1,1,Berlin,DEU,52.501345,13.410907,headr,web,2012,8,"[13.4109071, 52.5013449]"
3,3,San Mateo,USA,37.566879,-122.323895,Fixya,web,2013,30,"[-122.323895, 37.566879]"
5,5,Santa Clara,USA,37.760524,-122.387799,Fuzz,games_video,2011,6,"[-122.387799, 37.760524]"
6,6,Van Nuys,USA,40.650291,-74.294395,CollegeConvo,web,2010,2,"[-74.294395, 40.650291]"


### Se guarda en un json el dataframe

In [10]:
data.to_json('oficinas.json', orient="records", lines= True)

### Se crea la base de datos, colección y se asigna el índice

In [12]:
db_ofi = client['oficinas']

def create_db(archivo = 'oficinas'):
    '''
    Crea la base de y la coleeción para oficinas
    '''
    
    existe = os.path.isfile(archivo + '.json')
    if existe:
        os.system('mongoimport --db oficinas --collection oficinas --drop --file '+ archivo +'.json')
        db_ofi.oficinas.create_index([('coordenadas', GEOSPHERE )])
        print('Base de datos y collección creadas')
    else:
        raise ValueError('Error archivo no encontrado')
    
create_db()

Base de datos y collección creadas


### Se realiza la búsqueda de las oficinas más cercanas para cada registro

In [13]:
def buscar_cercanas(lng, lat, min_dist_m = 0, max_dist_m = 3000):
    '''
    devuelve las oficinas más cercanas a un punto dado
    '''
    
    nearLocation = {
        "lng": lng,
        "lat": lat
    }
    busqueda = db_ofi.oficinas.find({
        "coordenadas": {
         "$near": {
           "$geometry": {
              "type": "Point" ,
              "coordinates": [ nearLocation["lng"] , nearLocation["lat"] ]
           },
           '$minDistance': min_dist_m,
           "$maxDistance": max_dist_m, 
         }
       }
        
    },{'latitude': 1,
       'longitude': 1,
       'city': 1,
       'name': 1,
       'category_code': 1,
       'number_of_employees': 1,
       '_id': 0})
    
    return busqueda   

In [21]:
data['cercanas'] = data.apply(lambda x : list(buscar_cercanas(x['longitude'], x['latitude'])), axis = 1)

def ratios_sb(oficinas):
    '''
    calcula el ratio de grandes empresas respecto a startup
    '''
    startup, big = 0, 0
    for office in oficinas:
        if office['number_of_employees'] > 250:
            big += 1
        else:
            startup += 1
    if startup > 0 and big > 0:
        salida = big / startup
    else:
        salida = 100 
    return salida


### Se añade la columna num_offices y el ratio y se crea un dataframe para los puntos que tengan más de 5 oficinas a su alrededor

In [24]:
data['num_offices'] = data['cercanas'].apply(lambda x: len(x))
data['ratio'] = data['cercanas'].apply(ratios_sb)
datos_finales = data[(data.num_offices > 5)]
datos_finales.head()

,index,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees,coordenadas,cercanas,num_offices,ratio
0,0,New York,USA,40.757929,-73.985506,PeekYou,search,2012,20,"[-73.985506, 40.757929]","[{'city': 'New York', 'latitude': 40.757929, '...",6,1.0
18,18,New York,USA,40.744618,-73.987764,Yipit,web,2010,23,"[-73.987764, 40.744618]","[{'city': 'New York', 'latitude': 40.744618, '...",6,1.0
23,23,San Francisco,USA,37.805324,-122.405276,Indee,games_video,2010,3,"[-122.4052761, 37.8053241]","[{'city': 'San Francisco', 'latitude': 37.8053...",13,12.0
38,2,New York,USA,40.755716,-73.979247,Facebook,social,2004,5299,"[-73.9792469, 40.7557162]","[{'city': 'New York', 'latitude': 40.7557162, ...",6,1.0
39,3,San Francisco,USA,37.776805,-122.416924,Twitter,social,2006,1300,"[-122.4169244, 37.7768052]","[{'city': 'San Francisco', 'latitude': 37.7768...",14,100.0


### Se busca el ratio menor, ya que queremos la menor diferencia entre número de startup y grandes empresas

In [27]:
final = data.cercanas[(datos_finales['ratio'].idxmin())]
df_final = pd.DataFrame(final)
df_final.head(50)

,category_code,city,latitude,longitude,name,number_of_employees
0,search,New York,40.757929,-73.985506,PeekYou,20
1,social,New York,40.755716,-73.979247,Facebook,5299
2,software,New York,40.764577,-73.979901,Unison Technologies,30
3,web,New York,40.744618,-73.987764,Yipit,23
4,search,New York,40.742220,-74.004489,Google,28000
5,web,New York,40.741888,-74.004747,MLB Advanced Media,600


### Se calcula la latitud y longitud media respecto a las oficinas anteriores, para posicionar la nueva oficina

In [28]:
long_med = df_final['longitude'].mean()
lat_med = df_final['latitude'].mean()

nueva_ofi = ['games_video', 'New York', lat_med, long_med, '2 Game', 50]

df_final.loc[len(df_final)] = nueva_ofi
df_final.head(50)

,category_code,city,latitude,longitude,name,number_of_employees
0,search,New York,40.757929,-73.985506,PeekYou,20
1,social,New York,40.755716,-73.979247,Facebook,5299
2,software,New York,40.764577,-73.979901,Unison Technologies,30
3,web,New York,40.744618,-73.987764,Yipit,23
4,search,New York,40.742220,-74.004489,Google,28000
5,web,New York,40.741888,-74.004747,MLB Advanced Media,600
6,games_video,New York,40.751158,-73.990276,2 Game,50


### Se crea archivo para pasar a tableau

In [30]:
data.to_json('oficinas_tableau.json', orient="records", lines= True)

### Se muestra mapa con las localizaciones de todas las empresas 

In [33]:
mapa = folium.Map(location=[10, 0], tiles="openstreetmap", zoom_start=2)

for i in range(0,len(data)):
    if  data.iloc[i]['number_of_employees'] >= 250:
        folium.Marker([data.iloc[i]['latitude'], 
                       data.iloc[i]['longitude']], 
                      popup=data.iloc[i]['name'],
                      icon=folium.Icon(color='blue')).add_to(mapa)
    else:
        folium.Marker([data.iloc[i]['latitude'], 
                       data.iloc[i]['longitude']], 
                      popup=data.iloc[i]['name'],
                      icon=folium.Icon(color='green')).add_to(mapa)  
        
        
        
leyenda = '''
    <style type="text/css">
        #leyenda{
            position: fixed;
            z-index: 9999;
            font-size: 1em;
            background-color: #ffffff;
            color: #333333;
            bottom: 10px;
            right: 10px;
            padding: 10px;
            border: 1px solid #333333;
        }
        
        #leyenda .verde{
            color: #71af26;
        }
        
        #leyenda .azul{
            color: #36a5d6;
        }
    </style>
    
     <div id="leyenda">
        <i class="fa fa-map-marker fa-2x verde"></i> Startup <br>
        <i class="fa fa-map-marker fa-2x azul"></i> Big Company
      </div>
     '''
mapa.get_root().html.add_child(folium.Element(leyenda))
mapa.save('map-oficinas.html')


In [79]:
#mapa

### Mapa con la localización final de la oficina

In [77]:
loc_oficina = df_final[['latitude', 'longitude']][(df_final['name'] == '2 Game')]
loc_oficina.reset_index()

,index,latitude,longitude
0,6,40.751158,-73.990276


In [80]:
punto = [loc_oficina.iloc[0]['latitude'], loc_oficina.iloc[0]['longitude']]
mapa_final = folium.Map(location= punto, tiles="openstreetmap", zoom_start=15)

for i in range(0,len(df_final)):
    if  df_final.iloc[i]['name'] == '2 Game':
        folium.Marker([df_final.iloc[i]['latitude'], 
                       df_final.iloc[i]['longitude']], 
                      popup=df_final.iloc[i]['name'],
                      icon=folium.Icon(color='red')).add_to(mapa_final)
    else:
        folium.Marker([df_final.iloc[i]['latitude'], 
                       df_final.iloc[i]['longitude']], 
                      popup=df_final.iloc[i]['name'],
                      icon=folium.Icon(color='green')).add_to(mapa_final) 
        
mapa_final.save('map-final.html')        

In [78]:
#mapa_final